# Initial conditions in v2 routing
- Currently, v2 routing does not allow for initial flow conditions to be specified. Rather, an initial state of zero cms is assumed for all segments in the network
- To make v2 routing operational, we need a way to specify the initial conditions from a restart file. 
- Here, I am tinkering to develop such capability
- I will first experiment with reassigning column values in a numpy array
- Then I will try to create a new argument in the compute_network call (cython)

In [12]:
import os
import sys
import subprocess
import glob
import pathlib

# logic to accomodate Google Colab
try:
    import google.colab

    ENV_IS_CL = True

    root = pathlib.Path("/content/t-route").resolve()
    subprocess.run(
        [
            "git",
            "clone",
            "-b",
            "network-compute-notebook",
            "https://github.com/awlostowski-noaa/t-route.git",
        ]
    )

    subprocess.run("pip", "install", "geopandas")
    subprocess.run("pip", "install", "netcdf4")
    subprocess.run("pip", "install", "gif")

except:
    ENV_IS_CL = False
    root = pathlib.Path("..").resolve()

fortran_source_dir = os.path.join(
    root, "src", "fortran_routing", "mc_pylink_v00", "MC_singleSeg_singleTS"
)
sys.path.append(fortran_source_dir)
routing_v02_dir = os.path.join(root, "src", "python_routing_v02")
sys.path.append(routing_v02_dir)
sys.path.append(os.path.join(root, "src", "python_framework_v02"))
sys.path.append(os.path.join(root, "src", "python_framework_v01"))

COMPILE_cython = True
# ! This will definitely work in the Google Colab environment, though may not work
# in your local envioronment.
if COMPILE_cython:

    subprocess.run(
        ["gfortran", "varPrecision.f90", "-c", "-o", "var_precision.o", "-O3", "-fPIC"],
        cwd=fortran_source_dir,
    )
    subprocess.run(
        [
            "gfortran",
            "MCsingleSegStime_f2py_NOLOOP.f90",
            "-c",
            "-o",
            "mc_single_seg.o",
            "-O3",
            "-fPIC",
        ],
        cwd=fortran_source_dir,
    )
    subprocess.run(
        [
            "gfortran",
            "pyMCsingleSegStime_NoLoop.f90",
            "-c",
            "-o",
            "pymc_single_seg.o",
            "-O3",
            "-fPIC",
        ],
        cwd=fortran_source_dir,
    )

    for obj_file in glob.glob(os.path.join(fortran_source_dir, "*.o")):
        print(obj_file)
        subprocess.run(["cp", obj_file, routing_v02_dir])

    if ENV_IS_CL:
        numpy_I = "/usr/local/lib/python3.6/dist-packages/numpy/core/include"
        py_I = "/usr/include/python3.6"
        py_lib = "/usr/include/python3.6"

    else:
        bin_exe = pathlib.Path(sys.executable)
        virtual_env_path = bin_exe.parents[1]
        py_lib = os.path.join(virtual_env_path, "lib")
        numpy_I = os.path.join(
            py_lib, "python3.6", "dist-packages", "numpy", "core", "include"
        )
        py_I = os.path.join(virtual_env_path, "include", "python3.6m")

    subprocess.run(
        [
            "cython",
            "-3",
            "-v",
            "-p",
            "--line-directives",
            "-Wextra",
            "--cleanup",
            "3",
            "mc_reach.pyx",
        ],
        cwd=routing_v02_dir,
    )
    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-Wno-unused-result",
            "-Wsign-compare",
            "-DNDEBUG",
            "-g",
            "-fwrapv",
            "-O3",
            "-fno-strict-aliasing",
            "-Wall",
            "-Wstrict-prototypes",
            "-fPIC",
            "-I",
            numpy_I,
            "-I",
            py_I,
            "-c",
            "mc_reach.c",
            "-o",
            "mc_reach.o",
        ],
        cwd=routing_v02_dir,
    )
    subprocess.run(
        [
            "gcc",
            "-pthread",
            "-shared",
            "-L",
            py_lib,
            "var_precision.o",
            "mc_single_seg.o",
            "pymc_single_seg.o",
            "mc_reach.o",
            "-lgfortran",
            "-o",
            "mc_reach.cpython-36m-x86_64-linux-gnu.so",
        ],
        cwd=routing_v02_dir,
    )

# import scientific libraries
import time
import numpy as np
from operator import itemgetter
from itertools import chain, islice
from functools import partial
from joblib import delayed, Parallel
import random
import pandas as pd
import numpy as np
from scipy.stats import norm
import geopandas
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter("ignore")

# network/reach utilities & routing module
import nhd_network_utilities_v02 as nnu
import nhd_io
import nhd_network
import mc_reach
import network_dl

/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/MC_singleSeg_singleTS/mc_single_seg.o
/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/MC_singleSeg_singleTS/pymc_single_seg.o
/mnt/c/Users/awlostowski/Documents/national-water-center/t-route/src/fortran_routing/mc_pylink_v00/MC_singleSeg_singleTS/var_precision.o


In [13]:
'''
cpdef object compute_network(int nsteps, list reaches, dict connections, 
    const long[:] data_idx, object[:] data_cols, const float[:,:] data_values, 
    const float[:, :] qlat_values, 
    bint assume_short_ts=False):
    
    ...
    
    cdef float[:,::1] flowveldepth = np.zeros((data_idx.shape[0], nsteps * 3), dtype='float32')
    
'''

# number of segments in the network
nsegs = 100
# number of timesteps
nsteps = 200

# creates an array of zeros (nsegs rows, nsteps* 3 columns)
flowveldepth = np.zeros((nsegs, nsteps * 3), dtype = 'float32')

initial_flow = 100
initial_depth = 0.72
initial_vel = 1.2

flowveldepth[:, 0] = initial_flow
flowveldepth[:, 1] = initial_vel
flowveldepth[:, 2] = initial_depth


In [23]:
def sim(dx, dt, q_baseflow, q_pk, short_tstep_assumption):

    ################################################
    # Set-up model domain
    ################################################
    # SPACE
    L = 20 * (1000) # meters
    nsegs = round(L/dx)

    # TIME
    n_days = 6
    len_sim = n_days*(24*60*60) # seconds
    nts = round(len_sim/dt)


    ################################################
    # Create model parameter set
    ################################################
    link_start = 8815967 # arbitrary linkID for the top segment of the reach
    headwaters = [8815960, 8815961] # create two headwaters, one of which will accept lateral inflows

    # construct linkIDs and destination linkIDs
    linkID = list(range(link_start,
                        link_start + nsegs,
                        1))
    linkID = headwaters + linkID

    headwater_to = [link_start, link_start]
    toID = list(range(link_start + 1,
                      link_start + nsegs,
                      1))
    toID.append(0)
    toID = headwater_to + toID

    # link-distance lookup - what is the physical distance associated with each segment? 
    link_distance = {}
    for i, link in enumerate(linkID[2:],1):
        link_distance[i*dx] = link

    # build parameter DataFrame
    d = {'link': linkID,
          'to': toID,
          'dx': [dx] * (nsegs+2),
          'n': [0.028] * (nsegs+2),
          'ncc': [0.031] * (nsegs+2),
          's0': [0.0018] * (nsegs+2),
          'bw': [112] * (nsegs+2),
          'tw': [448] * (nsegs+2),
          'twcc': [623] * (nsegs+2),
          'MusK': [3600.0] * (nsegs+2),
          'MusX': [0.2] * (nsegs+2),
          'cs': [1.40] * (nsegs+2)}

    df = pd.DataFrame(data=d)
    df = df.set_index('link')
    df = df.sort_index()

    # replace downstreams for terminal segments
    df = nhd_io.replace_downstreams(df, 'to', 0)

    ################################################
    # Create lateral inflow data
    ################################################
    t_pk = 2*24*(60*60)
    width = 2*(60*60)


    x = np.linspace(0,len_sim, round(len_sim/dt))
    y = norm.pdf(x,t_pk,width)
    y_scale = y * q_pk/max(y) + q_baseflow

    # construct lateral inflow series
    ql = pd.DataFrame(0, index=linkID, columns=range(round(len_sim/dt)),dtype=np.float32)
    ql.iloc[0,:] = y_scale

    ################################################
    # Build network
    ################################################
    conn = nhd_network.extract_connections(df, 'to')
    rconn = nhd_network.reverse_network(conn)
    subnets = nhd_network.reachable_network(rconn, check_disjoint = False)

    # identify the segments in each subnetwork
    subreachable = nhd_network.reachable(rconn)

    # break each subnetwork into reaches
    subreaches = {}
    for tw, net in subnets.items():
        path_func = partial(nhd_network.split_at_junction, net)
        subreaches[tw] = nhd_network.dfs_decomposition(net, path_func)

    tail_water = max(linkID)
    reach = subreaches[tail_water]
    r = list(filter(None, chain.from_iterable(reach)))

    ################################################
    # Simulate network routing
    ################################################
    # add a dt column to the data DataFrame
    df["dt"] = dt

    # change variables to type float32, as expected by mc_reach.compute_network
    df = df.astype("float32")

    # prep parameter and lateral inflow data to be fed to routing model
    data_sub = df.loc[
        r, ["dt", "dx", "bw", "tw", "twcc", "n", "ncc", "cs", "s0"]
    ].sort_index()

    # create qlat dataset
    qlat_sub = ql.loc[r].sort_index()

    # compute the network routing, calculate (flow, depth, and velocity)
    results = []

    results.append(
        mc_reach.compute_network(
            nts,
            reach,
            subnets[tail_water],
            data_sub.index.values.astype("int64"),
            data_sub.columns.values,
            data_sub.values,
            qlat_sub.values.astype("float32"),
            assume_short_ts=short_tstep_assumption
        )
    )

    ################################################
    # re-format simulation output
    ################################################

    # create a multi-index DataFrame with flow, depth, and velocity simulations
    fdv_columns = pd.MultiIndex.from_product([range(nts), ["q", "v", "d"]])
    flowveldepth = pd.concat(
        [pd.DataFrame(d, index=i, columns=fdv_columns) for i, d in results], copy=False
    )
    flowveldepth = flowveldepth.sort_index()


    # create flow DataFrame
    flows = flowveldepth.loc[:, (slice(None), "q")]
    flows = flows.T.reset_index(level = [0,1])
    flows.rename(columns = {"level_0": "Timestep", "level_1": "Parameter"}, inplace = True)
    flows['Time (d)'] = ((flows.Timestep + 1) * dt)/(24*60*60)
    flows = flows.set_index('Time (d)')
    flows = flows.drop(["Timestep", "Parameter"], axis=1)
    
    forcing = flows.loc[:,flows.columns.min()]
    
    flows = flows.drop(headwaters, axis = 'columns')

    # create depth DataFrame
    depth = flowveldepth.loc[:, (slice(None), "d")]
    depth = depth.T.reset_index(level = [0,1])
    depth.rename(columns = {"level_0": "Timestep", "level_1": "Parameter"}, inplace = True)
    depth['Time (d)'] = ((depth.Timestep + 1) * dt)/(24*60*60)
    depth = depth.set_index('Time (d)')
    depth = depth.drop(["Timestep", "Parameter"], axis=1)
    depth = depth.drop(headwaters, axis = 'columns')
    
    # create vel DataFrame
    vel = flowveldepth.loc[:, (slice(None), "v")]
    vel = vel.T.reset_index(level = [0,1])
    vel.rename(columns = {"level_0": "Timestep", "level_1": "Parameter"}, inplace = True)
    vel['Time (d)'] = ((vel.Timestep + 1) * dt)/(24*60*60)
    vel = vel.set_index('Time (d)')
    vel = vel.drop(["Timestep", "Parameter"], axis=1)
    vel = vel.drop(headwaters, axis = 'columns')
    
    return flows, depth, vel, df, forcing

In [24]:
# simulate routing without initializing the flow
dx = 300
dt = 600
q_baseflow = 100 # cms
q_pk = 1 # cms
assume_short_timestep = False
            
flow, depth, vel, df, forcing = sim(dx, dt, q_baseflow, q_pk, assume_short_timestep)